# 1. Importing Libraries & Data

In [3]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

**Import Data**

In [4]:
X_train = pd.read_csv('./project_data/x_train.csv', index_col = 'Claim Identifier')
X_val = pd.read_csv('./project_data/x_val.csv', index_col = 'Claim Identifier')
y_train = pd.read_csv('./project_data/y_train.csv', index_col = 'Claim Identifier')
y_val = pd.read_csv('./project_data/y_val.csv', index_col = 'Claim Identifier')

test = pd.read_csv('./project_data/test_treated.csv', index_col = 'Claim Identifier')

# 2. Modelling

<a href="#top">Top &#129033;</a>

**Baseline Model**

In [ ]:
model = 
model.fit(X_train, y_train)

In [ ]:
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

In [ ]:
print(classification_report(y_train, train_pred))
print(classification_report(y_val, val_pred))


## 2.1 Hyperparameter Tuning

<a href="#top">Top &#129033;</a>

## 2.2 Voting Ensemble

<a href="#top">Top &#129033;</a>

In [1]:
from sklearn.ensemble import VotingClassifier

# 3. Final Predictions

<a href="#top">Top &#129033;</a>

In [ ]:
test['Claim Injury Type'] = model.predict(test)

Map Predictions to Original Values

In [ ]:
label_mapping = {
    1: "1. CANCELLED",
    2: "2. NON-COMP",
    3: "3. MED ONLY",
    4: "4. TEMPORARY",
    5: "5. PPD SCH LOSS",
    6: "6. PPD NSL",
    7: "7. PTD",Check each category inside target

test['Claim Injury Type'].value_counts() 
    8: "8. DEATH"
}

test['Claim Injury Type'] = test['Claim Injury Type'].replace(label_mapping)

Check each category inside the target

In [ ]:
test['Claim Injury Type'].value_counts() 

# 5. Export

<a href="#top">Top &#129033;</a>

**Select Columns for predictions**

In [ ]:
predictions = test['Claim Injury Type']

**Export**

In [ ]:
name = None

In [ ]:
predictions.to_csv(f'./predictions/{name}.csv')